In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer, FunctionTransformer,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K


In [2]:
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# Check for GPUs
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

if gpus:
    for gpu in gpus:
        print(f"GPU Name: {gpu.name}")
        
        # Get details of each GPU
        try:
            details = tf.config.experimental.get_device_details(gpu)
            print(f"GPU Device Details: {details}")
        except RuntimeError as e:
            print(f"Failed to get details for GPU {gpu.name}: {e}")
    
    try:
        # Set memory growth for each GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set successfully.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs found.")


TensorFlow version: 2.10.0
Num GPUs Available:  1
GPU Name: /physical_device:GPU:0
GPU Device Details: {'device_name': 'NVIDIA GeForce RTX 3060', 'compute_capability': (8, 6)}
Memory growth set successfully.


In [3]:
import tensorflow as tf

# Enable device placement logging
tf.debugging.set_log_device_placement(True)

In [4]:
from tensorflow.keras.backend import clear_session
clear_session()

In [5]:
reduced_loaded_data_x= np.load(r'E:\projects\paper\myself\data_processing\x_data_non_downsampled.npz')
reduced_loaded_data_y = np.load(r'E:\projects\paper\myself\data_processing\y_data_non_downsampled.npz')

x= reduced_loaded_data_x['x']
y= reduced_loaded_data_y['x']



In [6]:
# Define the directory where models are saved
model_dir = r'E:\projects\paper\myself\testing\cnn'
metrics_csv_path = 'metrics.csv'
plots_dir = r'E:\projects\paper\myself\testing\plots'

In [7]:
x

array([[ 0.05343904,  0.10471563,  0.09451785, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02075644, -0.0404014 , -0.03602667, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00218729,  0.00407564,  0.00339912, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.00942281,  0.01889487,  0.01717605, ...,  0.        ,
         0.        ,  0.        ],
       [-0.05081851, -0.09910698, -0.08860061, ...,  0.        ,
         0.        ,  0.        ],
       [-0.05121048, -0.09565088, -0.07949052, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [8]:
x_shape=x.shape
x_shape

(917, 413760)

In [9]:
y

array([5, 5, 3, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 3, 3, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 3, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 5, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 5, 5, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,
       3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5,

In [10]:
y_shape=y.shape
y_shape

(917,)

In [11]:
unique_classes = np.unique(y)

# Number of unique classes
num_classes = len(unique_classes)
num_classes

6

In [12]:
def stratified_split(X, y, test_size=0.2, val_size=0.25, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=random_state, stratify=y_train)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [13]:
X_train, X_val, X_test, y_train, y_val, y_test = stratified_split(x, y)


In [14]:
f=lambda x : pd.Series(x).value_counts()


In [15]:
f(y_train)

2    475
4     22
3     21
5     13
0     10
1      8
Name: count, dtype: int64

In [16]:
f(y_val)

2    159
4      8
3      7
5      5
0      3
1      2
Name: count, dtype: int64

In [17]:
f(y_test)

2    159
3      7
4      7
5      5
1      3
0      3
Name: count, dtype: int64

# testing

In [18]:

# # Ensure the plots directory exists
# os.makedirs(plots_dir, exist_ok=True)

# def evaluate_model(model, x_test, y_test):
#     # Get predictions
#     y_pred = model.predict(x_test)
    
#     # Convert predictions to class labels
#     y_pred_labels = np.argmax(y_pred, axis=1)
    
#     # Calculate metrics
#     accuracy = accuracy_score(y_test, y_pred_labels)
    
#     # Calculate precision, recall, f1 score for different averaging methods
#     precision_micro = precision_score(y_test, y_pred_labels, average='micro')
#     precision_macro = precision_score(y_test, y_pred_labels, average='macro')
#     precision_weighted = precision_score(y_test, y_pred_labels, average='weighted')
    
#     recall_micro = recall_score(y_test, y_pred_labels, average='micro')
#     recall_macro = recall_score(y_test, y_pred_labels, average='macro')
#     recall_weighted = recall_score(y_test, y_pred_labels, average='weighted')
    
#     f1_micro = f1_score(y_test, y_pred_labels, average='micro')
#     f1_macro = f1_score(y_test, y_pred_labels, average='macro')
#     f1_weighted = f1_score(y_test, y_pred_labels, average='weighted')
    
#     # Calculate ROC AUC score
#     y_test_one_hot = tf.keras.utils.to_categorical(y_test)
#     roc_auc = roc_auc_score(y_test_one_hot, y_pred, multi_class='ovr')
    
#     # Calculate loss
#     loss = log_loss(y_test, y_pred)
    
#     return {
#         'accuracy': accuracy,
#         'precision_micro': precision_micro,
#         'precision_macro': precision_macro,
#         'precision_weighted': precision_weighted,
#         'recall_micro': recall_micro,
#         'recall_macro': recall_macro,
#         'recall_weighted': recall_weighted,
#         'f1_micro': f1_micro,
#         'f1_macro': f1_macro,
#         'f1_weighted': f1_weighted,
#         'roc_auc': roc_auc,
#         'loss': loss
#     }

# def plot_metrics_from_csv(csv_path):
#     metrics_df = pd.read_csv(csv_path)
#     metrics_df.set_index('Model', inplace=True)
    
#     for metric in ['Accuracy', 'Precision (Micro)', 'Precision (Macro)', 'Precision (Weighted)',
#                    'Recall (Micro)', 'Recall (Macro)', 'Recall (Weighted)',
#                    'F1 Score (Micro)', 'F1 Score (Macro)', 'F1 Score (Weighted)',
#                    'ROC AUC', 'Loss']:
#         plt.figure(figsize=(12, 8))
#         plt.plot(metrics_df.index, metrics_df[metric], marker='o', label=metric)
#         plt.xlabel('Model')
#         plt.ylabel(metric)
#         plt.title(f'{metric} for Different Models')
#         plt.xticks(rotation=45)
#         plt.legend()
#         plt.grid(True)
#         plt.tight_layout()
#         plt.savefig(f'plots/{metric}.png')
#         plt.close()


In [19]:

# def main(x_test, y_test):
    # x_test = np.reshape(x_test, (-1, 1, x_test.shape[1]))
    # print(f"Shape after reshaping: {x_test.shape}")
    # x_test = tf.convert_to_tensor(x_test)
    # print(f"Shape after tensor conversion: {x_test.shape}")
    # y_test=tf.convert_to_tensor(y_test)
    # print(f"Shape after tensor conversion: {y_test.shape}")

#     # Clear TensorFlow session
#     K.clear_session()

#     metrics_data = {
#         'Model': [],
#         'Accuracy': [],
#         'Precision (Micro)': [],
#         'Precision (Macro)': [],
#         'Precision (Weighted)': [],
#         'Recall (Micro)': [],
#         'Recall (Macro)': [],
#         'Recall (Weighted)': [],
#         'F1 Score (Micro)': [],
#         'F1 Score (Macro)': [],
#         'F1 Score (Weighted)': [],
#         'ROC AUC': [],
#         'Loss': []
#     }

#     for model_file in os.listdir(model_dir):
#         if model_file.endswith('.h5'):
#             model_name = model_file.replace('.h5', '')
#             print(f"Evaluating model: {model_name}")
            
#             try:
#                 # Load the model
#                 model_path = os.path.join(model_dir, model_file)
#                 model = load_model(model_path)
#                 print("shape",model.input_shape)

#                 # Evaluate the model
#                 metrics = evaluate_model(model, x_test, y_test)
                
#                 # Append metrics to the data dictionary
#                 metrics_data['Model'].append(model_name)
#                 metrics_data['Accuracy'].append(metrics['accuracy'])
#                 metrics_data['Precision (Micro)'].append(metrics['precision_micro'])
#                 metrics_data['Precision (Macro)'].append(metrics['precision_macro'])
#                 metrics_data['Precision (Weighted)'].append(metrics['precision_weighted'])
#                 metrics_data['Recall (Micro)'].append(metrics['recall_micro'])
#                 metrics_data['Recall (Macro)'].append(metrics['recall_macro'])
#                 metrics_data['Recall (Weighted)'].append(metrics['recall_weighted'])
#                 metrics_data['F1 Score (Micro)'].append(metrics['f1_micro'])
#                 metrics_data['F1 Score (Macro)'].append(metrics['f1_macro'])
#                 metrics_data['F1 Score (Weighted)'].append(metrics['f1_weighted'])
#                 metrics_data['ROC AUC'].append(metrics['roc_auc'])
#                 metrics_data['Loss'].append(metrics['loss'])

#             except Exception as e:
#                 print(f"Error evaluating model {model_name}: {e}")
#             finally:
#                 K.clear_session()

#     # Convert metrics data to DataFrame
#     metrics_df = pd.DataFrame(metrics_data)
    
#     # Append to CSV or create a new one if it doesn't exist
#     if os.path.exists(metrics_csv_path):
#         existing_metrics_df = pd.read_csv(metrics_csv_path)
#         combined_metrics_df = pd.concat([existing_metrics_df, metrics_df])
#         combined_metrics_df.to_csv(metrics_csv_path, index=False)
#     else:
#         metrics_df.to_csv(metrics_csv_path, index=False)
    
#     # Plot metrics from the updated CSV file
#     plot_metrics_from_csv(metrics_csv_path)

# if __name__ == "__main__":
#     main(X_test, y_test)

In [20]:
class_names =['Bronchiectasis', 'Bronchiolitis', 'COPD', 'Healthy', 'Pneumonia','URTI']


In [21]:

def evaluate_model(model, x_test, y_test):
    # Perform model evaluation
    # Ensure you adapt this function based on your actual evaluation metrics and format
    eval_results = model.evaluate(x_test, y_test, verbose=0)
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, accuracy_score, log_loss
    
    # Compute metrics
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_classes, average='macro')
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(y_test, y_pred_classes, average='micro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test, y_pred_classes, average='weighted')
    roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')
    loss = log_loss(y_test, y_pred)

    # Store per-class metrics
    class_metrics = {}
    for i, class_name in enumerate(class_names):
        class_metrics[str(i)] = {
            'precision': precision_recall_fscore_support(y_test, y_pred_classes, average=None)[0][i],
            'recall': precision_recall_fscore_support(y_test, y_pred_classes, average=None)[1][i],
            'f1-score': precision_recall_fscore_support(y_test, y_pred_classes, average=None)[2][i]
        }

    return {
        'accuracy': accuracy,
        'precision_micro': precision_micro,
        'precision_macro': precision,
        'precision_weighted': precision_weighted,
        'recall_micro': recall_micro,
        'recall_macro': recall,
        'recall_weighted': recall_weighted,
        'f1_micro': f1_micro,
        'f1_macro': f1,
        'f1_weighted': f1_weighted,
        'roc_auc': roc_auc,
        'loss': loss,
        'class_metrics': class_metrics
    }

def plot_metrics_from_csv(csv_path):
    metrics_df = pd.read_csv(csv_path)
    metrics_df.set_index('Model', inplace=True)
    
    # Plot overall metrics
    for metric in ['Accuracy', 'Precision (Micro)', 'Precision (Macro)', 'Precision (Weighted)',
                   'Recall (Micro)', 'Recall (Macro)', 'Recall (Weighted)',
                   'F1 Score (Micro)', 'F1 Score (Macro)', 'F1 Score (Weighted)',
                   'ROC AUC', 'Loss']:
        plt.figure(figsize=(12, 8))
        plt.plot(metrics_df.index, metrics_df[metric], marker='o', label=metric)
        plt.xlabel('Model')
        plt.ylabel(metric)
        plt.title(f'{metric} for Different Models')
        plt.xticks(rotation=45)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'plots/{metric}.png')
        plt.close()

    # Plot per-class metrics
    for class_name in class_names:
        for metric in ['Precision', 'Recall', 'F1 Score']:
            plt.figure(figsize=(12, 8))
            plt.plot(metrics_df.index, metrics_df[f'{metric} ({class_name})'], marker='o', label=f'{metric} ({class_name})')
            plt.xlabel('Model')
            plt.ylabel(f'{metric} ({class_name})')
            plt.title(f'{metric} for {class_name} Across Models')
            plt.xticks(rotation=45)
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(f'plots/{metric}_{class_name}.png')
            plt.close()


In [22]:

def main(x_test, y_test):
    # Ensure x_test is reshaped and converted to a TensorFlow tensor
    # x_test = np.reshape(x_test, (-1, 1, x_test.shape[1]))
    # print(f"Shape after reshaping: {x_test.shape}")
    x_test = tf.convert_to_tensor(x_test)
    print(f"Shape after tensor conversion: {x_test.shape}")
    y_test=tf.convert_to_tensor(y_test)
    print(f"Shape after tensor conversion: {y_test.shape}")



    # Clear TensorFlow session
    K.clear_session()

    # Initialize metrics data structure
    metrics_data = {
        'Model': [],
        'Accuracy': [],
        'Precision (Micro)': [],
        'Precision (Macro)': [],
        'Precision (Weighted)': [],
        'Recall (Micro)': [],
        'Recall (Macro)': [],
        'Recall (Weighted)': [],
        'F1 Score (Micro)': [],
        'F1 Score (Macro)': [],
        'F1 Score (Weighted)': [],
        'ROC AUC': [],
        'Loss': []
    }

    # Add columns for per-class metrics
    for class_name in class_names:
        metrics_data[f'Precision ({class_name})'] = []
        metrics_data[f'Recall ({class_name})'] = []
        metrics_data[f'F1 Score ({class_name})'] = []

    # Iterate over the model files and evaluate each
    for model_file in os.listdir(model_dir):
        if model_file.endswith('.h5'):
            model_name = model_file.replace('.h5', '')
            print("\n\n\n\n\n\n\n")
            print(f"Evaluating model: {model_name}")

            try:
                # Load the model
                model_path = os.path.join(model_dir, model_file)
                model = load_model(model_path)

                # Print model input shape for debugging
                print(f"Expected input shape for model {model_name}: {model.input_shape}")

                # Ensure x_test matches the model's expected input shape
                if len(x_test.shape) != len(model.input_shape):
                    print(f"Reshaping x_test from {x_test.shape} to match the model's expected input shape.")
                    # Reshape x_test here if necessary
                    # For example, if the model expects a 4D input for a CNN:
                    # x_test = np.expand_dims(x_test, axis=-1)  # Adding channel dimension

                # Evaluate the model
                metrics = evaluate_model(model, x_test, y_test)

                # Append overall metrics
                metrics_data['Model'].append(model_name)
                metrics_data['Accuracy'].append(metrics['accuracy'])
                metrics_data['Precision (Micro)'].append(metrics['precision_micro'])
                metrics_data['Precision (Macro)'].append(metrics['precision_macro'])
                metrics_data['Precision (Weighted)'].append(metrics['precision_weighted'])
                metrics_data['Recall (Micro)'].append(metrics['recall_micro'])
                metrics_data['Recall (Macro)'].append(metrics['recall_macro'])
                metrics_data['Recall (Weighted)'].append(metrics['recall_weighted'])
                metrics_data['F1 Score (Micro)'].append(metrics['f1_micro'])
                metrics_data['F1 Score (Macro)'].append(metrics['f1_macro'])
                metrics_data['F1 Score (Weighted)'].append(metrics['f1_weighted'])
                metrics_data['ROC AUC'].append(metrics['roc_auc'])
                metrics_data['Loss'].append(metrics['loss'])

                # Append per-class metrics
                for class_name in class_names:
                    class_idx = class_names.index(class_name)
                    metrics_data[f'Precision ({class_name})'].append(
                        metrics['class_metrics'][str(class_idx)]['precision']
                    )
                    metrics_data[f'Recall ({class_name})'].append(
                        metrics['class_metrics'][str(class_idx)]['recall']
                    )
                    metrics_data[f'F1 Score ({class_name})'].append(
                        metrics['class_metrics'][str(class_idx)]['f1-score']
                    )

            except Exception as e:
                print(f"Error evaluating model {model_name}: {e}")
            finally:
                K.clear_session()

    # Convert metrics data to DataFrame
    metrics_df = pd.DataFrame(metrics_data)

    # Append to CSV or create a new one if it doesn't exist
    if os.path.exists(metrics_csv_path):
        try:
            existing_metrics_df = pd.read_csv(metrics_csv_path)
            combined_metrics_df = pd.concat([existing_metrics_df, metrics_df], axis=0, ignore_index=True)
            combined_metrics_df.to_csv(metrics_csv_path, index=False)
        except pd.errors.EmptyDataError:
            metrics_df.to_csv(metrics_csv_path, index=False)
    else:
        metrics_df.to_csv(metrics_csv_path, index=False)

    # Plot metrics from the updated CSV file
    # plot_metrics_from_csv(metrics_csv_path)


if __name__ == "__main__":
    main(X_test, y_test)


Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Shape after tensor conversion: (184, 413760)
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Shape after tensor conversion: (184,)








Evaluating model: cnnrobust
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0

c:\Users\ahamed\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ahamed\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ahamed\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVar

c:\Users\ahamed\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ahamed\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ahamed\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [23]:
plot_metrics_from_csv(metrics_csv_path)